In [ ]:
from create_solutions import get_next_guess, make_all_possibilities

In [ ]:
real_all_pos = make_all_possibilities(path="all_possibilites.json")

In [ ]:
history = {
    '48-12=36': tuple(int(i) for i in "20200100"),
}

get_next_guess(real_all_pos=real_all_pos, history=history)